<h1> Import Data </h1>

In [1]:
%matplotlib inline
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn import grid_search, linear_model as lm, cross_validation as cv, tree, preprocessing as prep

iowa = pd.read_csv('IowaLiquor.csv',parse_dates=[0])
iowa.columns = ['Date',
                'StoreNo',
                'City',
                'Zip',
                'CountyNo',
                'County',
                'CategoryNo',
                'Category',
                'VendorNo',
                'ItemNo',
                'Item',
                'Volume',
                'Cost',
                'Retail',
                'Quantity',
                'TotSales',
                'TotVolume',
                'TotVolumeGals']
iowa

,Date,StoreNo,City,Zip,CountyNo,County,CategoryNo,Category,VendorNo,ItemNo,Item,Volume,Cost,Retail,Quantity,TotSales,TotVolume,TotVolumeGals
0,2015-11-04,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,$4.50,$6.75,12,$81.00,9.00,2.38
1,2016-03-02,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,Tin Cup,750,$13.75,$20.63,2,$41.26,1.50,0.40
2,2016-02-11,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,19067,Jim Beam,1000,$12.59,$18.89,24,$453.36,24.00,6.34
3,2016-02-03,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,395,59154,1800 Ultimate Margarita,1750,$9.50,$14.25,6,$85.50,10.50,2.77
4,2015-08-18,3654,BELMOND,50421,99.0,Wright,1031080.0,VODKA 80 PROOF,297,35918,Five O'clock Vodka,1750,$7.20,$10.80,12,$129.60,21.00,5.55
5,2015-04-20,2569,CEDAR RAPIDS,52402,57.0,Linn,1041100.0,AMERICAN DRY GINS,205,31473,New Amsterdam Gin,1750,$13.32,$19.98,6,$119.88,10.50,2.77
6,2015-08-05,2596,OTTUMWA,52501,90.0,Wapello,1051010.0,AMERICAN GRAPE BRANDIES,85,52806,Korbel Brandy,750,$6.66,$9.99,3,$29.97,2.25,0.59
7,2015-06-25,3456,CLEAR LAKE,50428,17.0,Cerro Gordo,1012100.0,CANADIAN WHISKIES,65,10628,Canadian Club Whisky,1750,$15.75,$23.63,2,$47.26,3.50,0.92
8,2016-01-04,4757,BONDURANT,50035,77.0,Polk,1032080.0,IMPORTED VODKA,370,34006,Absolut Swedish Vodka 80 Prf,750,$11.49,$17.24,4,$68.96,3.00,0.79
9,2015-11-10,4346,SHELLSBURG,52332,6.0,Benton,1081315.0,CINNAMON SCHNAPPS,65,82610,Dekuyper Hot Damn!,1000,$7.62,$11.43,2,$22.86,2.00,0.53


<h1> Format Data </h1>

In [5]:
#Change dollar columns to a number
def dollartonum(col, df=iowa):
    df[col] = df[[col]].replace('[\$,]','',regex=True).astype(float)

dollartonum('TotSales')
dollartonum('Retail')
dollartonum('Cost')

def int2(x):
    try:
        return int(x)
    except ValueError:
        pass
    return 0

#Change CountyNo and CategoryNo to categorical variables
iowa['CountyNo'] = iowa['CountyNo'].apply(float).apply(int2).astype('category')
iowa['CategoryNo'] = iowa['CategoryNo'].apply(float).apply(int2).astype('category')

<h1> New Variables </h1>

In [7]:
import datetime as dt
#Add the Year variable
iowa['Year'] = pd.DatetimeIndex(iowa['Date']).year
#Quarter
iowa['Quarter'] = pd.DatetimeIndex(iowa['Date']).quarter
#TotalMargin
iowa['TotMargin'] = (iowa['Retail']-iowa['Cost'])*iowa['Quantity']
#PricePerLitre
iowa['PricePerLitre'] = iowa['TotSales']/iowa['Quantity']

<h1> Alcohol Categories </h1>

In [26]:
#Get the alcohol type of each drink
catypes = {'WHISKIES':['WHISKIES','WHISKY','WHISKEY','SCOTCH','BOURBON'],
           'BRANDIES':['BRANDIES'],
           'VODKA':['VODKA'],
           'GINS':['GINS','GIN'],
           'SCHNAPPS':['SCHNAPPS'],
           'TEQUILA':['TEQUILA'],
           'RUM':['RUM'],
           'LIQUEURS':['LIQUEURS',
                      'LIQUEUR',
                      'AMARETTO',
                      'CREME',
                      'ROCK',
                      'ANISETTE',
                      'DISTILLED'],
           'BEER':['BEER']
           }

#reshape the dictionary
catypes2 = dict()
for catname, keywordlist in catypes.iteritems():
    for keyword in keywordlist:
        catypes2[keyword] = catname

def cats(cat):
    try:
        for word in cat.split():
            if word in catypes2.keys():
                #print word
                return catypes2[word]
    except AttributeError:
        return "OTHER"
    return "OTHER"

iowa['Category2'] = iowa['Category'].apply(cats)

In [148]:
#aggregating sales by this new category
sales_cats = iowa.groupby(['StoreNo',
              'Quarter',
              'Category2'])[['TotSales']].agg(sum)

sales_cats2 = sales_cats.unstack().fillna(0)
sales_cats2.columns = sales_cats2.columns.droplevel()
sales_cats2['HighestRevCat'] = sales_cats2.idxmax(axis=1)
sales_cats2

#below is the BAD way, but kept for the record
#sales_highestcat = sales_cats.reset_index().sort_values(['StoreNo','Quarter','TotSales'],ascending=[1,1,0]).groupby(['StoreNo','Quarter']).first()
#sales_highestcat = sales_highestcat.drop('TotSales',1).rename(columns={'Category2':'HighestRevCat'})
#pd.merge(sales_cats2, sales_highestcat,left_index=True,right_index=True)

Category2          BEER  BRANDIES     GINS  LIQUEURS    OTHER       RUM  \
StoreNo Quarter                                                           
2106    1          0.00   1194.96  1118.49   9777.42  1125.30  15945.84   
        2          0.00    444.48  1330.20   2826.38  1029.72   2356.96   
        3          0.00     98.64  1104.90   2991.13   360.48   5356.44   
        4          0.00    195.48   367.08   3289.42  1636.80   6226.72   
2113    1          0.00    142.26   113.23    329.29     0.00    590.56   
        2          0.00     28.49     0.00     45.97     0.00    408.69   
        3          0.00    127.40    68.64    141.13     0.00    204.75   
        4          0.00    142.26   120.74     78.01    18.28     71.68   
2130    1         99.00   3944.94  1934.88   4947.70   241.92  10948.83   
        2          0.00    439.08   514.68   2217.87   217.44   3517.62   
        3          0.00    395.48   555.24   1595.26   261.36   3980.88   
        4          0.00   1854.00   604.50   2024.88   658.56   3324.60   
2152    1          0.00     32.43     7.13    122.39    28.65    457.54   
        2          0.00     48.42     0.00      7.13     7.26    293.82   
        3          0.00     18.73     6.35    275.22     0.00    657.18   
        4          0.00     40.46     0.00    160.91    29.25    443.62   
2178    1         22.86    795.22   135.12   1282.23   366.27   2203.60   
        2          0.00   1107.06    71.76     87.88   185.82    656.40   
        3          0.00    102.00   142.60    339.34     0.00   1218.94   
        4          0.00    128.04   116.88    335.16    13.86   1432.26   
2190    1         11.43  12372.02  3041.17   2376.88  3509.51   2265.88   
        2          0.00   8354.72  3093.87   1549.94   264.43   1465.88   
        3          0.00   2803.91  1149.15    866.80    97.78   1267.93   
        4          0.00   8420.56  1568.52    948.38   914.85   4014.94   
2191    1         99.00    593.99   937.75   6049.80   302.52  16107.45   
        2         99.00    560.81   701.33   3676.81   294.34  13320.05   
        3        297.00    144.82   108.85   1789.73   369.63   1126.19   
        4          0.00    240.84   861.48   4468.11   642.97   2079.93   
2200    1          0.00    252.56    31.48    612.72   142.64   1405.43   
        2          0.00     16.44   144.18    301.40   180.60   1137.05   
...                 ...       ...      ...       ...      ...       ...   
5224    1          0.00    225.84   274.16    216.96   680.64    431.76   
5225    1          0.00      0.00     0.00      0.00   216.18     81.00   
5226    1          0.00    595.44   141.84      0.00     0.00      0.00   
5227    1          0.00    517.21    14.95     47.43     0.00    167.67   
5229    1          0.00      0.00     0.00     33.39     0.00     45.00   
5230    1          0.00    171.67   235.01     38.24   391.05    183.05   
5232    1          0.00      0.00    31.47      0.00     0.00     45.00   
5233    1          0.00      0.00     0.00      0.00     0.00    207.00   
5234    1          0.00      0.00     0.00      0.00     0.00    162.00   
5236    1          0.00    234.96    97.56    548.74   186.30    796.25   
5237    1          0.00    413.28     0.00      0.00     0.00     86.40   
5240    1          0.00      0.00     0.00     29.34     0.00    238.50   
5247    1          0.00      0.00     0.00      0.00     0.00      0.00   
9001    1          0.00    601.68     0.00      0.00     0.00      0.00   
        2          0.00      0.00     0.00    800.40     0.00      0.00   
        3          0.00    184.56   221.40      0.00     0.00    630.00   
        4          0.00    337.56     0.00      0.00     0.00      0.00   
9002    1          0.00      0.00  1164.48      0.00     0.00      0.00   
        2          0.00      0.00     0.00      0.00     0.00      0.00   
        3          0.00      0.00   327.36      0.00     0.00      0.00   
        4          0.00  

<h1> Aggregates </h1>

In [36]:
daterange = lambda x: ((np.max(x)-np.min(x)) / np.timedelta64(1, 'D')).astype(int)

def newmode(series):
    try:
        return series.mode()[0]
    except IndexError:
        print "jghgjhg", series

def aggsales(year=0,quarter=0):
    
    if year != 0 and quarter != 0:
        df = iowa[(iowa['Year']==year) & (iowa['Quarter']==quarter)]
    elif year != 0:
        df = iowa[(iowa['Year']==year)]
    else:
        df = iowa
    
    dfout = pd.pivot_table(df,
                   index=['Quarter',
                          'StoreNo',
                          'City',
                          'Zip',
                          'CountyNo',
                          'County'],
                   values=['TotSales',
                           'Date',
                           'Quantity',
                           'TotMargin',
                           'TotVolume'
                          ],
                   aggfunc = {'TotSales':{'TotSales':np.sum},
                              'TotMargin':{'TotMargin':np.sum},
                              'TotVolume':{'TotVolume':np.sum},
                              'Quantity':{'TotQty':np.sum,
                                          'TotNoSales':len},
                              'Date':{'DateRange':daterange,
                                      'DaysSold':pd.Series.nunique}
                             })
    
    dfout.columns = dfout.columns.droplevel()
    dfout['AvgBottlePrice'] = dfout['TotSales']/dfout['TotQty']
    dfout['AvgSaleAmount'] = dfout['TotSales']/dfout['TotNoSales']
    dfout['AvgPricePerLitre'] = dfout['TotSales']/dfout['TotQty']
    dfout['AvgMarginPercent'] = dfout['TotMargin']/dfout['TotSales']
    
    dfout['DaysSold'] = dfout['DaysSold'].astype(int)
    dfout['DateRange'] = dfout['DateRange'].astype(int)
    
    return dfout

sales_2015q1 = aggsales(2015,1)
#sales_2016q1 = aggsales(2016,1)
#sales_2015 = aggsales(2015)
#sales_agg = aggsales()
sales_2015q1

DateRange  \
Quarter StoreNo City            Zip   CountyNo County                     
1       2106    CEDAR FALLS     50613 7        Black Hawk            77   
        2113    GOWRIE          50543 94       Webster               77   
        2130    WATERLOO        50703 7        Black Hawk            77   
        2152    ROCKWELL        50469 17       Cerro Gordo           77   
        2178    WAUKON          52172 3        Allamakee             77   
        2190    DES MOINES      50314 77       Polk                  84   
        2191    KEOKUK          52632 56       Lee                   84   
        2200    SAC CITY        50583 81       Sac                   77   
        2205    CLARINDA        51632 73       Page                  84   
        2228    WINTERSET       50273 61       Madison               84   
        2233    SPIRIT LAKE     51360 30       Dickinson             84   
        2238    ALTOONA         50009 77       Polk                  70   
        2248    DES MOINES      50312 77       Polk                  77   
        2285    IOWA CITY       52240 52       Johnson               77   
        2290    STORM LAKE      50588 11       Buena Vista           77   
        2327    CORNING         50841 2        Adams                 84   
        2353    CRESCO          52136 45       Howard                84   
        2367    SHENANDOAH      51601 73       Page                  84   
        2413    DUBUQUE         52001 31       Dubuque               83   
        2445    OAKLAND         51560 78       Pottawattamie         84   
        2448    INDIANOLA       50125 91       Warren                84   
        2459    GUTHRIE CENTER  50115 39       Guthrie               77   
        2460    HAMPTON         50441 35       Franklin              84   
        2465    DUBUQUE         52003 31       Dubuque               83   
        2475    WEST BEND       50597 55       Kossuth               84   
        2478    ALTOONA         50009 77       Polk                  84   
        2487    ANAMOSA         52205 53       Jones                 84   
        2498    ARLINGTON       50606 33       Fayette               84   
        2500    AMES            50010 85       Story                 77   
        2501    AMES            50010 85       Story                 77   
...                                                                 ...   
        5086    SIOUX CITY      51106 97       Woodbury              42   
        5088    HOSPERS         51238 84       Sioux                 70   
        5089    MISSOURI VALLEY 51555 43       Harrison              77   
        5090    MUSCATINE       52761 70       Muscatine             73   
        5091    MUSCATINE       52761 70       Muscatine             10   
        5092    WEST DES MOINES 50265 77       Polk                  63   
        5093    LE CLAIRE       52753 82       Scott                 63   
        5094    CEDAR RAPIDS    52401 57       Linn                  56   
        5096    NORTH LIBERTY   52317 52       Johnson               28   
        5097    CEDAR RAPIDS    52402 57       Linn                  56   
        5098    DES MOINES      50311 77       Polk                  46   
        5099    ANKENY          50023 77       Polk                  40   
        5100    DAVENPORT       52802 82       Scott                 42   
        5101    WATERLOO        50701 7        Black Hawk            42   
        5102    MOUNT VERNON    52314 57       Linn                  53   
        5103    MUSCATINE       52761 70       Muscatine              0   
        5104    ARNOLD'S PARK   51331 30       Dickinson              0   
        5105    NORTH LIBERTY   52317 52       Johnson               21   
        5106    DEWITT          52742 23       Clinton               21   
        5108    BETTENDORF      52722 82       Scott                 28   
        5112    Inwood          51240 60       Lyon                  28   
        5113

In [ ]:
treeReg = tree.DecisionTreeRegressor()
linReg = lm.LinearRegression()
TSReg = lm.TheilSenRegressor()
RANReg = lm.RANSACRegressor()

In [ ]:
clf = grid_search.GridSearchCV(ols,{'max_depth':range(1,10)})

clf.fit(train_X,train_Y)
clf.best_estimator_.max_depth